In [18]:
# imports
import pandas as pd
import cdsapi
import os
import zipfile
import glob
import shutil
import pathlib
import numpy as np

In [19]:
# getting list of all IPCC models in the 2021 physical science IPCC sixth report

# read in the csv file i made with the list of models
def readModels():
    df = pd.read_csv(
        '/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/climate_models.csv')

    # reduce to those within the likely climate sensitivity range
    df_likely = df.loc[(df['ECS °C'] >= 2.5) & (df['ECS °C'] <= 4)]
    likely_models = df_likely['Model_IPCC'].reset_index( drop=True).replace('-', '_', regex=True).str.lower()  # create list of models
    return likely_models

In [48]:
# Defining all the options for monthly bioclim variables
def setData():

    pathways = ['historical', 'ssp1_2_6',  'ssp2_4_5','ssp5_8_5']
    variables = ['daily_maximum_near_surface_air_temperature', 'daily_minimum_near_surface_air_temperature','precipitation']
    periods = ['2021-01-01/2040-12-31', '2041-01-01/2060-12-31', '2061-01-01/2080-12-31', '2081-01-01/2100-12-31']
    countries = {'Mexico': [35, -120, 10,-80,], 'Colombia': [15, -80, -5,-65,], 'Mexico_and_Texas':[38, -120, 10,-80,] }
    temporal_resolution = 'monthly'

    '''
    # dummy data
    pathways = ['ssp1_2_6']
    variables = ['daily_maximum_near_surface_air_temperature','daily_minimum_near_surface_air_temperature', 'precipitation']
    periods = ['2021-01-01/2040-12-31']
    countries = {'Mexico': [35, -120, 10, -80, ],'Colombia': [15, -80, -5,-65,] }
    temporal_resolution = 'monthly'
    '''
    
    #separating locations and names for later use
    location_ranges = list(countries.values())
    location_names = list(countries.keys())
    return pathways, variables, periods, countries, temporal_resolution, location_ranges, location_names
    
    
 

# function to initialise query
def initialiseQuery():
    return {
    'temporal_resolution': 'monthly',
    'experiment': 'null',
    'level': 'single_levels',
    'variable': 'null',
    'model': 'null',
    'date': 'null',
    'format': 'zip',
    'area': 'null',
}

# function to set the attributes to retrieve
def setQuery(attributes, SSP, variable, model, region, years):
    attributes['experiment'] = SSP
    attributes['variable'] = variable
    attributes['model'] = model
    attributes['area'] = region
    attributes['date'] = years

#creating an empty dataframe to store which models were used for each variable
def createSummaryDF(query, likely_models):
    df_columns = list(query.keys()) + ['country'] + likely_models.to_list() + ['count_yes']
    for i in ['format', 'level', 'model', 'area']:
        df_columns.remove(i)
    df_models_used = pd.DataFrame(columns = df_columns)
    return df_models_used, df_columns

# a function adding information to the Dataframe
def createInfoForDF(df_models_used,country, years, SSP, variable,models_used):
    models_used = models_used.to_list()
    row_contents= ['monthly', SSP, variable, years, country] + models_used + [models_used.count('yes')]
    row_to_df = dict(zip(df_columns, row_contents))
    return df_models_used.append(row_to_df, ignore_index=True)

# creating function to unzip and move the relevant files to my harddrive under a different name
def unzip_and_move(zip_path, temporal_resolution, pathway, variable, model, country, years):
    dates = '_' + years[11:15]
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(os.getcwd() + '/download')
    new_path = '/Volumes/TOSHIBA EXT/' + temporal_resolution + '/' + country +  '/'+ dates+ '/' + pathway + '/' + variable + '/' + model + '.nc'
    if not os.path.isdir('/Volumes/TOSHIBA EXT/' + temporal_resolution + '/' + country +  '/'+ dates+ '/' + pathway + '/' + variable + '/'):
        os.makedirs('/Volumes/TOSHIBA EXT/' + temporal_resolution + '/' + country  +'/'+ dates+ '/' + pathway + '/' + variable + '/')
    netcdf_path = os.getcwd() + '/' + glob.glob('*/*.nc')[0]
    shutil.move(netcdf_path, new_path)
    shutil.rmtree(os.getcwd() + '/download')


#get what the country name is from the region that is being considered
def getCountry(region):
    return location_names[location_ranges.index(region)]

def downloadData(likely_models):
    #for region in location_ranges:
    region = location_ranges[2] #change depending on the country that you wish to look at
    global df_columns
    df_models_used, df_columns = createSummaryDF(query, likely_models)
    for variable in variables:
        for SSP in pathways:  # ssp stands for shared socioeconomic pathway
            if SSP != 'historical':
                for years in periods:
                    models_used = likely_models.copy().squeeze()
                    for model in likely_models:
                        setQuery(query, SSP, variable, model, region, years)
                        try:
                            print(model)
                            c = cdsapi.Client()

                            c.retrieve(
                                'projections-cmip6',
                                query,
                                'download.zip')

                            zip_path = os.getcwd() + '/download.zip'
                            
                            temporal_resolution = 'monthly'
                            country = getCountry(region)
                            unzip_and_move(zip_path, temporal_resolution, SSP, variable, model, country, years)
                            print(SSP + model + ': completed')
                            models_used = models_used.replace(model, 'yes')
                        except:
                            models_used = models_used.replace(model, 'no')
                            print(SSP + model + ': failed')

                    country = getCountry(region)
                    df_models_used = createInfoForDF(df_models_used, country, years, SSP, variable,models_used)
            
            else:
                years = '2001-01-01/2020-12-31'
                models_used = likely_models.copy().squeeze()
                for model in likely_models:
                    setQuery(query, SSP, variable, model, region, years)
                    try:
                        c = cdsapi.Client()

                        c.retrieve(
                            'projections-cmip6',
                            query,
                            'download.zip')

                        zip_path = os.getcwd() + '/download.zip'
                        
                        temporal_resolution = 'monthly'
                        country = getCountry(region)
                        unzip_and_move(zip_path, temporal_resolution, SSP, variable, model, country, years)
                        print(SSP + model + ': completed')
                        models_used = models_used.replace(model, 'yes')
                    except:
                        models_used = models_used.replace(model, 'no') #to be used in the df_models_used
                        print(SSP + model + ': failed')
                
                country = getCountry(region)
                df_models_used = createInfoForDF(df_models_used, country, years, SSP, variable,models_used)
    df_models_used.to_csv('/Volumes/TOSHIBA EXT/monthly/' + country + '/data_frame_of_models.csv', index = False)





In [21]:
def main():
  likely_models = readModels()
  global pathways, variables, periods, countries, temporal_resolution, location_ranges, location_names
  pathways, variables, periods, countries, temporal_resolution, location_ranges, location_names = setData()
  global query
  query = initialiseQuery()
  downloadData(likely_models)
  


In [49]:
main()

cnrm_cm6_1


2022-04-11 12:49:20,534 INFO Welcome to the CDS
2022-04-11 12:49:20,540 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2022-04-11 12:49:20,695 INFO Request is completed
2022-04-11 12:49:20,696 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data3/adaptor.esgf_wps.retrieve-1649669327.9870224-2320-18-9c5cfbed-4f36-4b29-bae9-aca32d47f31f.zip to download.zip (537.9K)
2022-04-11 12:49:21,389 INFO Download rate 776.7K/s
2022-04-11 12:49:21,538 INFO Welcome to the CDS
2022-04-11 12:49:21,545 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2022-04-11 12:49:21,639 INFO Request is queued


ssp1_2_6cnrm_cm6_1: completed
cesm2_waccm_fv2


2022-04-11 12:49:22,708 INFO Request is failed
2022-04-11 12:49:22,709 ERROR Message: an internal error occurred processing your request
2022-04-11 12:49:22,710 ERROR Reason:  No matching data for request {'experiment': 'ssp126', 'model': 'CESM2-WACCM-FV2', 'temporal_resolution': 'mon', 'variable': 'tasmax'}
2022-04-11 12:49:22,712 ERROR   Traceback (most recent call last):
2022-04-11 12:49:22,713 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
2022-04-11 12:49:22,714 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2022-04-11 12:49:22,715 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2022-04-11 12:49:22,716 ERROR       result = proc(context, *context.args, **context.kwargs)
2022-04-11 12:49:22,717 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 124, in __call__
2022-04-11 12:49:22,718 ERROR       return p(*args, **

ssp1_2_6cesm2_waccm_fv2: failed
cnrm_esm2_1


...

2022-04-11 12:49:50,964 INFO Download rate 4.7M/s   
2022-04-11 12:49:51,109 INFO Welcome to the CDS
2022-04-11 12:49:51,113 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2022-04-11 12:49:51,193 INFO Request is queued


ssp1_2_6cnrm_cm6_1_hr: completed
cams_csm1_0


2022-04-11 12:49:52,263 INFO Request is failed
2022-04-11 12:49:52,263 ERROR Message: an internal error occurred processing your request
2022-04-11 12:49:52,264 ERROR Reason:  No matching data for request {'experiment': 'ssp126', 'model': 'CAMS-CSM1-0', 'temporal_resolution': 'mon', 'variable': 'tasmin'}
2022-04-11 12:49:52,265 ERROR   Traceback (most recent call last):
2022-04-11 12:49:52,265 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
2022-04-11 12:49:52,266 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2022-04-11 12:49:52,268 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2022-04-11 12:49:52,269 ERROR       result = proc(context, *context.args, **context.kwargs)
2022-04-11 12:49:52,270 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 124, in __call__
2022-04-11 12:49:52,271 ERROR       return p(*args, **kwar

ssp1_2_6cams_csm1_0: failed


/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_1561/2175409282.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return df_models_used.append(row_to_df, ignore_index=True)
